In [81]:
import pandas as pd    # ура скачаем все что можно
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
import plotly.express as px
from geopy.geocoders import Nominatim
from time import sleep
from geopy.exc import GeocoderTimedOut

In [82]:
# Спарсим surf coffee (адреса и метро)

url = "https://www.surfcoffee.ru/surf-coffee-stores#rec76974741"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")


restaurants = soup.find_all("div", class_="t786__title t-name t-name_xl js-product-name") # тут парсим все серфы

for rest in restaurants:
    address = rest.find("div", class_="t786__descr t-descr t-descr_xxs")

    address_text = address.text.strip() if address else "Нет адреса"

restaurants_clean = []  # тут на сайте нашли, что NEO есть в названии последнего мск серфа, через поиск по слову убедились, что больше такого нет

for i in restaurants:
    restaurants_clean.append(i.text)
    if 'NEO' in i.text:
        break              # получили список мск серфов


restaurants_clean = [rest.strip().replace('®', '').replace('\xa0', ' ') for rest in restaurants_clean]


#print(len(restaurants_clean))   # 112 серфов в мск
restaurants_clean

['SOLYANKA × SURF COFFEE',
 'ARBAT × SURF COFFEE',
 'SECRET SPOT × SURF COFFEE',
 'LIPS × SURF COFFEE',
 'G-SPOT × SURF COFFEE',
 'MAYAK × SURF COFFEE',
 'SHORE × SURF COFFEE',
 'DREAM × SURF COFFEE',
 'ANTHILL X SURF COFFEE',
 'CHANGE X SURF COFFEE',
 'PKRVK X SURF COFFEE',
 'TAKEOFF X SURF COFFEE',
 'SPORT X SURF COFFEE',
 'URBAN JUNGLE X SURF COFFEE',
 'BELKA X SURF COFFEE',
 'REVOLUTION X SURF COFFEE',
 'RED VILLAGE X SURF COFFEE',
 'TRAVEL X SURF COFFEE',
 'FLAT X SURF COFFEE',
 'STUDIO X SURF COFFEE',
 'AMEN X SURF COFFEE',
 'SHINE X SURF COFFEE',
 'FLACON X SURF COFFEE',
 'EAST WEST X SURF COFFEE',
 'HIGHWAY X SURF COFFEE',
 'REPUBLIC X SURF COFFEE',
 'AVIAPARK X SURF COFFEE',
 'SABABA X SURF COFFEE',
 'AKADEMIK X SURF COFFEE',
 'OASIS X SURF COFFEE',
 'PI X SURF COFFEE',
 'LOVELY X SURF COFFEE',
 'FLOW × SURF COFFEE',
 'YOUTH × SURF COFFEE',
 'ARMA × SURF COFFEE',
 'PURE× SURF COFFEE',
 'OGNI × SURF COFFEE',
 'STARTUP × SURF COFFEE',
 'LE FORT × SURF COFFEE',
 'WINGS × SURF COF

In [83]:
# Спарсим surf coffee (адреса и метро)


address = soup.find_all("div", class_="t786__descr t-descr t-descr_xxs")

address[1:113:]
address_clean = []
for i in address[1:113:]:
    address_clean.append(i.text)
address_clean # получили адреса

# Убираем \n по краям и знак ®
address_clean = [addr.strip().replace('®', '') for addr in address_clean]


metro_list = []

# Регулярка для поиска метро (ищет "м. [название]")
metro_pattern = re.compile(r'м\.\s*([\w-]+)')

for address in address_clean:
    match = metro_pattern.search(address)
    if match:
        metro_list.append(match.group(1))  # Добавляем название метро
    else:
        metro_list.append(pd.NA)  # Если метро нет, добавляем NaN

print(metro_list) # как выяснили это все было ради 8 станций:(

print(address_clean)

['Китай-Город', 'Арбатская', 'Шаболовская', 'Академическая', 'Тургеневская', 'Маяковская', 'Мякинино', <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, 'Новокузнецкая', <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, '3', <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>]
['Москва, м. Китай-Город, ул. Солянка 1/2', 'Москва, м. Арбатская, ул. Новый Арбат 7а', 'Москва, м. Шаболовская, ул. Конный переулок д.12', 'Москва, м. Академическая, ул. Дмитрия Ульянова 4к1', 'Москва, м. Тургеневская, ул. Мясницкая 16', 'Москва, м. Маяков

In [84]:
# Сделаем датафрейм c surf coffee
address_clean = [re.sub(r"Москва(?=[^ ,])", "Москва ", address) for address in address_clean]
df = pd.DataFrame({'Названия': restaurants_clean, 'Адрес': address_clean, 'Метро': metro_list})
display(df)

,Названия,Адрес,Метро
0,SOLYANKA × SURF COFFEE,"Москва, м. Китай-Город, ул. Солянка 1/2",Китай-Город
1,ARBAT × SURF COFFEE,"Москва, м. Арбатская, ул. Новый Арбат 7а",Арбатская
2,SECRET SPOT × SURF COFFEE,"Москва, м. Шаболовская, ул. Конный переулок д.12",Шаболовская
3,LIPS × SURF COFFEE,"Москва, м. Академическая, ул. Дмитрия Ульянова...",Академическая
4,G-SPOT × SURF COFFEE,"Москва, м. Тургеневская, ул. Мясницкая 16",Тургеневская
...,...,...,...
107,GRAMMY X SURF COFFEE,Москва Варшавское ш. вл. 150б/1,<NA>
108,SUNDAY X SURF COFFEE,Москва Авиаторов 1,<NA>
109,BOVE X SURF COFFEE,Москва Академика Сахарова 7,<NA>
110,SRETENKA X SURF COFFEE,Москва ул. Сретенка 27с1,<NA>


In [85]:
geolocator = Nominatim(user_agent="coffee_map")

# Создаём новые столбцы для координат
df["Latitude"] = None
df["Longitude"] = None

# Проходим по всем адресам и получаем координаты
for i, row in df.iterrows():
    try:
        location = geolocator.geocode(row["Адрес"])
        if location:
            df.at[i, "Latitude"] = location.latitude
            df.at[i, "Longitude"] = location.longitude
    except Exception as e:
        print(f"Ошибка с адресом {row['Адрес']}: {e}")
    sleep(1)


# Строим карту
fig = px.scatter_mapbox(
    df,
    lat="Latitude",
    lon="Longitude",
    text="Названия",
    zoom=10,
    center={"lat": 55.7558, "lon": 37.6173},  # Центр Москвы
    mapbox_style="open-street-map"
)

Ошибка с адресом Москва Ленинградский пр-т 36 стр. 38: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0+%D0%9B%D0%B5%D0%BD%D0%B8%D0%BD%D0%B3%D1%80%D0%B0%D0%B4%D1%81%D0%BA%D0%B8%D0%B9+%D0%BF%D1%80-%D1%82+36+%D1%81%D1%82%D1%80.+38&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [86]:
url2 = "https://luchshie-dostavki.ru/restaurant/surf-coffee-x-element_73343/menu" # нашли какой-то сайт с доставкой из серфа, потому что у них а сайте цен нет
headers = {"User-Agent": "Mozilla/5.0"}

response2 = requests.get(url2, headers=headers)
soup2 = BeautifulSoup(response2.text, "html.parser")


dishes = soup2.find_all("div", class_="h6 mt-1") # тут парсим все серфы
prices = soup2.find_all("div", class_="h6 text-dark")

dishes_clean = []
prices_clean = []

for di in dishes:
    dishes_clean.append(di.text)

for pr in prices:
    prices_clean.append(pr.text)

dishes_clean = [item.strip() for item in dishes_clean]
prices_clean = [item.strip() for item in prices_clean]
prices_clean = [int(re.sub(r"\D+", "", price)) for price in prices_clean] # убрали "руб." и сделали числами


df_surf_menu = pd.DataFrame({'Блюдо': dishes_clean, 'Цена (руб.)': prices_clean})
display(df_surf_menu)

from google.colab.data_table import DataTable # ГПТ подсказал как полностью посмотреть на датафрейм

DataTable(df_surf_menu)

,Блюдо,Цена (руб.)
0,Капучино Grand,380
1,Пончик лимонно-маковый,250
2,Капучино Medium,345
3,Шу фисташковое,255
4,Батончик Фундук в молочной глазури,150
...,...,...
120,Батончик ореховый с маслом CBD,260
121,Дрип Кофе Kenya Tatu,450
122,Кофе зерновой Kenya Okland,900
123,Кофе зерновой Burundi Nabassa,900


,Блюдо,Цена (руб.)
0,Капучино Grand,380
1,Пончик лимонно-маковый,250
2,Капучино Medium,345
3,Шу фисташковое,255
4,Батончик Фундук в молочной глазури,150
...,...,...
120,Батончик ореховый с маслом CBD,260
121,Дрип Кофе Kenya Tatu,450
122,Кофе зерновой Kenya Okland,900
123,Кофе зерновой Burundi Nabassa,900


In [87]:
# Найдем среднюю цену напитков 0 1   52-62 77-116 - индексы напитков
indexes = [0, 1] + list(range(52, 63)) + list(range(77, 117))
average_surf_price_beverage = float(np.mean([prices_clean[i] for i in indexes]))
average_surf_price_beverage  # средняя цена напитка в surf_cofee - 391 рубль

389.1509433962264

In [15]:
# Спарсим stars (адреса и метро) На следующий день оно нормально не работало...за 15 минут не пофиксилось, поэтому просто скопирую лсит адресов

#url3 = "https://stars-coffee.ru/#menu"  # Укажи правильный URL
#headers = {"User-Agent": "Mozilla/5.0"}

#response3 = requests.get(url3, headers=headers)
#soup3 = BeautifulSoup(response.text, "html.parser")


#restaurants_stars = soup3.find_all("div", class_="t390__descr t-descr t-descr_xs")


#restaurants_clean_stars = []

#for rest in restaurants_stars:
    #address_text = rest.text.strip() if rest else "Нет адреса"
    #restaurants_clean_stars.append(address_text)


data_stars = {
    "Адрес": [
        "11, стр. 1", "ТЦ Щука, Щукинская ул., 42", "Ходынский бульвар, 4, ТЦ АвиаПарк, 2 этаж",
        "Бутырский Вал ул., 10", "ул. Гашека, 6", "Ленинградский проспект, 39 стр 79, БЦ Скай Лайт",
        "Новослободская ул., 4", "Лазоревый проезд 1а корп 2", "Проезд Аэропорта, д.",
        "Ул Тверская 8 к 1 с 5", "Ходынский бульвар д.3", "Кутузовский пр",
        "Пресненская набережная 2, 1 этаж", "Киевское шоссе 23-й км, 1", "ул. Покрышкина, 4",
        "Марксистская ул., 3", "Кутузовский проспект, 57", "проспект Вернадского, 6",
        "Чистопрудный бульвар, 1А", "Таганская улица, 1с1", "Кулакова 20",
        "Комсомольский проспект д.28, МДМ", "Ленинский пр-т., 109", "Каширское ш., 61",
        "ул. Ленинская Слобода, 26", "Мичуринский пр., 27", "Москва, ул. Проспект мира, 40",
        "Садовая-Самот", "ул. Долгоруковская 7", "Шереметьевская ул., 20", "ВТБ Арена –",
        "36, 1 уровень", "г. Москва, Правды ул., 26", "г. Москва, ул. Арбат 39-41",
        "Правобережная ул., 1Б", "М.О., Мытищинский р-н, ш. Алтуфьевское, 1-й км",
        "проспект Мира, 211к2", "ул. Земляной Вал, 33",
        "Москва, Пресненская наб, д. 10, блок В", "пл. Киевского вок", "3 этаж", "пл. Киевского вок"
    ],
    "Названия": ["Stars Coffee"] * 42
}

df_stars = pd.DataFrame(data_stars)
display(df_stars)


,Адрес,Названия
0,"11, стр. 1",Stars Coffee
1,"ТЦ Щука, Щукинская ул., 42",Stars Coffee
2,"Ходынский бульвар, 4, ТЦ АвиаПарк, 2 этаж",Stars Coffee
3,"Бутырский Вал ул., 10",Stars Coffee
4,"ул. Гашека, 6",Stars Coffee
5,"Ленинградский проспект, 39 стр 79, БЦ Скай Лайт",Stars Coffee
6,"Новослободская ул., 4",Stars Coffee
7,Лазоревый проезд 1а корп 2,Stars Coffee
8,"Проезд Аэропорта, д.",Stars Coffee
9,Ул Тверская 8 к 1 с 5,Stars Coffee


In [36]:
# Не бейте...на сайте меню было одной строчкой кода (как я понял это картинка, я не мог спарсить цены), поэтому загрузил картнку в гпт и попросил сделать df
data_stars = {
    "Блюдо": [
        "Капучино", "Латте", "Раф Ванилла", "Мокка", "Мокка Белый Шоколад",
        "Голден Макиато", "Флэт Уайт", "Эспрессо 46 мл", "Фильтр-кофе", "Горячий Шоколад",
        "Латте Шоколадный Мусс с Миндалем", "Латте Лимонный Курд с Шоколадом",
        "Латте Сливочная Фисташка", "Бамбл Маття", "Бамбл Кофе", "Эспрессо-тоник",
        "Маття-тоник", "Американо Пряный Крем"
    ],
    "Маленький (350 мл)": [
        320, 365, 370, 420, 395, 405, 390, 230, 255, 370,
        410, 410, 410, 410, 410, 385, 385, 335
    ],
    "Средний (450 мл)": [
        345, 385, 390, 430, 420, 430, None, None, 265, 390,
        430, 430, 430, 435, 435, 400, 400, 360
    ],
    "Большой (550 мл)": [
        365, 415, 420, 455, 445, 445, None, None, 285, 415,
        445, 445, 445, 455, 455, 430, 430, 380
    ]
}

df_stars_beverage = pd.DataFrame(data_stars)

df_stars_beverage['Средняя стоимость'] = df_stars_beverage.iloc[:, 1:].mean(axis=1, skipna=True)

display(df_stars_beverage) # норм

stars_average_price = df_stars_beverage['Средняя стоимость'].mean()
print(stars_average_price) # ~ 389руб. - средняя стоимость напитка с страс

,Блюдо,Маленький (350 мл),Средний (450 мл),Большой (550 мл),Средняя стоимость
0,Капучино,320,345.0,365.0,343.333333
1,Латте,365,385.0,415.0,388.333333
2,Раф Ванилла,370,390.0,420.0,393.333333
3,Мокка,420,430.0,455.0,435.000000
4,Мокка Белый Шоколад,395,420.0,445.0,420.000000
5,Голден Макиато,405,430.0,445.0,426.666667
6,Флэт Уайт,390,NaN,NaN,390.000000
7,Эспрессо 46 мл,230,NaN,NaN,230.000000
8,Фильтр-кофе,255,265.0,285.0,268.333333
9,Горячий Шоколад,370,390.0,415.0,391.666667


389.25925925925924


In [16]:
geolocator = Nominatim(user_agent="coffee_map")


df_stars["Latitude"] = None
df_stars["Longitude"] = None

for i, row in df_stars.iterrows():
    try:
        location = geolocator.geocode(row["Адрес"])
        if location:
            df_stars.at[i, "Latitude"] = location.latitude
            df_stars.at[i, "Longitude"] = location.longitude
    except Exception as e:
        print(f"Ошибка с адресом {row['Адрес']}: {e}")
    sleep(1)

Ошибка с адресом 11, стр. 1: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=11%2C+%D1%81%D1%82%D1%80.+1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом проспект Мира, 211к2: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82+%D0%9C%D0%B8%D1%80%D0%B0%2C+211%D0%BA2&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [18]:
# Skuratov также перебил из другого файла
data_skuratov = {
    "Адрес": [
        "Верхняя Радищевская, 19/3 ст2", "Кутузовский пр-т, 36 ст39", "Калашный переулок, 5",
        "Мира, 26 ст1", "Александра Невского, 1", "Благовещенский переулок, 1Б",
        "Нижняя Красносельская, 35 стр 49", "Рочдельская, 15 стр.1", "Мясницкая, 13 стр.2",
        "Комсомольский пр-кт, 24/1", "Хлебозавод — Новодмитровская, 1с27",
        "1-я Ямского Поля, д. 1, к. 1", "Усиевича, 12", "г. Химки, микрорайон ИКЕА, корпус 2"
    ],
    "Названия": ["Skuratov"] * 14
}

df_skuratov = pd.DataFrame(data_skuratov)
display(df_skuratov)

,Адрес,Названия
0,"Верхняя Радищевская, 19/3 ст2",Skuratov
1,"Кутузовский пр-т, 36 ст39",Skuratov
2,"Калашный переулок, 5",Skuratov
3,"Мира, 26 ст1",Skuratov
4,"Александра Невского, 1",Skuratov
5,"Благовещенский переулок, 1Б",Skuratov
6,"Нижняя Красносельская, 35 стр 49",Skuratov
7,"Рочдельская, 15 стр.1",Skuratov
8,"Мясницкая, 13 стр.2",Skuratov
9,"Комсомольский пр-кт, 24/1",Skuratov


In [19]:
# coordinates
df_skuratov["Latitude"] = None
df_skuratov["Longitude"] = None

for i, row in df_skuratov.iterrows():
    try:
        location = geolocator.geocode(row["Адрес"])
        if location:
            df_skuratov.at[i, "Latitude"] = location.latitude
            df_skuratov.at[i, "Longitude"] = location.longitude
    except Exception as e:
        print(f"Ошибка с адресом {row['Адрес']}: {e}")
    sleep(1)

display(df_skuratov)

,Адрес,Названия,Latitude,Longitude
0,"Верхняя Радищевская, 19/3 ст2",Skuratov,None,None
1,"Кутузовский пр-т, 36 ст39",Skuratov,None,None
2,"Калашный переулок, 5",Skuratov,55.755107,37.60147
3,"Мира, 26 ст1",Skuratov,45.029551,40.713402
4,"Александра Невского, 1",Skuratov,55.775252,37.592302
5,"Благовещенский переулок, 1Б",Skuratov,55.767314,37.597679
6,"Нижняя Красносельская, 35 стр 49",Skuratov,None,None
7,"Рочдельская, 15 стр.1",Skuratov,55.75683,37.559773
8,"Мясницкая, 13 стр.2",Skuratov,55.768937,37.645436
9,"Комсомольский пр-кт, 24/1",Skuratov,None,None


In [50]:
# сайт скуратова динамический, я не захотел подключать селениум ради 10-15 цен
data_skuratov_beverage = {
    "Блюдо": [
        "ЛАТТЕ", "ФЛЕТ УАЙТ", "МАТЧА ЛАТТЕ", "КАКАО",
        "СКУРАТОВ ЭСПРЕССО", "СЕЗОННЫЙ ЭСПРЕССО",
        "ФИЛЬТР-КОФЕ", "ФИЛЬТР-КОФЕ С ВИШНЕЙ", "Капучино", "Ванильный раф", "Американо"
    ],
    "250 мл": [None, 350, None, 450, 300, 300, 300, 430, 350, None, 300],
    "350 мл": [350, None, 410, 500, None, None, 350, 480, 400, 500, None],
    "450 мл": [450, None, 460, 550, None, None, 400, 530, 450, 550, None],
}

df_skuratov_beverage = pd.DataFrame(data_skuratov_beverage)
df_skuratov_beverage['Средняя стоимость'] = df_skuratov_beverage.iloc[:, 1:].mean(axis=1, skipna=True)
display(df_skuratov_beverage)

def lowercase_names(df, column_name="Напиток"):
    df[column_name] = df[column_name].str.lower()
    return df

,Блюдо,250 мл,350 мл,450 мл,Средняя стоимость
0,ЛАТТЕ,NaN,350.0,450.0,400.0
1,ФЛЕТ УАЙТ,350.0,NaN,NaN,350.0
2,МАТЧА ЛАТТЕ,NaN,410.0,460.0,435.0
3,КАКАО,450.0,500.0,550.0,500.0
4,СКУРАТОВ ЭСПРЕССО,300.0,NaN,NaN,300.0
5,СЕЗОННЫЙ ЭСПРЕССО,300.0,NaN,NaN,300.0
6,ФИЛЬТР-КОФЕ,300.0,350.0,400.0,350.0
7,ФИЛЬТР-КОФЕ С ВИШНЕЙ,430.0,480.0,530.0,480.0
8,Капучино,350.0,400.0,450.0,400.0
9,Ванильный раф,NaN,500.0,550.0,525.0


In [91]:
# Ариадна прислала места для возможной аренды, посмотрим как много конкурнетов около них
rent_places = ['Москва, ЦАО, р-н Тверской, 1-я Тверская-Ямская ул., 25С1',
'Москва, ЦАО, р-н Пресненский, Звенигородское ш., 18/20к1',
'Москва, ЦАО, р-н Замоскворечье, Люсиновская ул., 36С1',
'Москва, ЦАО, р-н Арбат, ул. Новый Арбат, 19',
'Москва, ЦАО, р-н Тверской, ул. Бутырский Вал, 68',
'Москва, ЦАО, р-н Арбат, ул. Арбат, 30/3С1',
'Москва, ЦАО, р-н Замоскворечье, Валовая ул., 21К125',
'Москва, ЦАО, р-н Пресненский, Большая Грузинская ул., 56',
'Москва, ЦАО, р-н Пресненский, Звенигородское ш., 18/20к1',
'Москва, ЦАО, р-н Тверской, 1-я Тверская-Ямская ул., 27'
]
rent_places_df = pd.DataFrame({'Адрес': rent_places})
rent_places_df["latitude"] = None
rent_places_df["longitude"] = None

#for i, row in rent_places_df.iterrows():
   # try:
   #     location = geolocator.geocode(row["Адрес"])
  #      if location:
  #          rent_places_df.at[i, "Latitude"] = location.latitude
  #          rent_places_df.at[i, "Longitude"] = location.longitude
  #  except Exception as e:
  #      print(f"Ошибка с адресом {row['Адрес']}: {e}") # не находит:(.
 #   sleep(1)

rent_latitude = [55.775586,
    55.764323,
    55.723328,
    55.751948,
    55.78932,
    55.749314,
    55.730101,
    55.771297,
    55.764323,
    55.775793
]
rent_longitude = [37.586006, 37.556119, 37.62122, 37.589293, 37.588853, 37.590219, 37.630939, 37.584038, 37.556119, 37.585556]

rent_places_df["latitude"] = rent_latitude
rent_places_df["longitude"] = rent_longitude
rent_places_df["radius_km"] = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
display(rent_places_df)

,Адрес,latitude,longitude,radius_km
0,"Москва, ЦАО, р-н Тверской, 1-я Тверская-Ямская...",55.775586,37.586006,0.5
1,"Москва, ЦАО, р-н Пресненский, Звенигородское ш...",55.764323,37.556119,0.5
2,"Москва, ЦАО, р-н Замоскворечье, Люсиновская ул...",55.723328,37.621220,0.5
3,"Москва, ЦАО, р-н Арбат, ул. Новый Арбат, 19",55.751948,37.589293,0.5
4,"Москва, ЦАО, р-н Тверской, ул. Бутырский Вал, 68",55.789320,37.588853,0.5
5,"Москва, ЦАО, р-н Арбат, ул. Арбат, 30/3С1",55.749314,37.590219,0.5
6,"Москва, ЦАО, р-н Замоскворечье, Валовая ул., 2...",55.730101,37.630939,0.5
7,"Москва, ЦАО, р-н Пресненский, Большая Грузинск...",55.771297,37.584038,0.5
8,"Москва, ЦАО, р-н Пресненский, Звенигородское ш...",55.764323,37.556119,0.5
9,"Москва, ЦАО, р-н Тверской, 1-я Тверская-Ямская...",55.775793,37.585556,0.5


In [23]:

df_stars["Сеть"] = "Stars Coffee"
df_skuratov["Сеть"] = "Skuratov"
df["Сеть"] = "Surf Coffee"


df_all = pd.concat([df_stars, df_skuratov, df])


fig = px.scatter_mapbox(
    df_all,
    lat="Latitude",
    lon="Longitude",
    text="Названия",
    color="Сеть",  # Цвета по сетям кофеен
    zoom=10,
    center={"lat": 55.7558, "lon": 37.6173},  # Центр Москвы
    mapbox_style="carto-positron"
)

fig.show()

In [100]:
fig = go.Figure()   # круги помог сделать гпт

fig.add_trace(go.Scattermapbox(
    lat=df_all["Latitude"],
    lon=df_all["Longitude"],
    mode="markers+text",
    marker=dict(size=10, color="blue"),
    text=df_all["Сеть"],
    textposition="top center",
    name="Города (df_all)"
))


fig.add_trace(go.Scattermapbox(
    lat=rent_places_df["latitude"],
    lon=rent_places_df["longitude"],
    mode="markers+text",
    marker=dict(size=10, color="red"),
    text=rent_places_df["Адрес"],
    textposition="top center",
    name="Арендуемые места (rent_places_df)"
))


def create_circle(lat, lon, radius_km):
    radius_deg = radius_km / 111

    lats = []
    lons = []
    for angle in np.linspace(0, 360, 100):
        lats.append(lat + radius_deg * np.sin(np.radians(angle)))
        lons.append(lon + radius_deg * np.cos(np.radians(angle)))

    lats.append(lats[0])
    lons.append(lons[0])

    return lats, lons

for index, row in rent_places_df.iterrows():
    lats, lons = create_circle(row["latitude"], row["longitude"], row["radius_km"])

    fig.add_trace(go.Scattermapbox(
        lat=lats,
        lon=lons,
        mode="lines",
        line=dict(width=2, color="green"),
        fill="toself",
        fillcolor="rgba(0, 255, 0, 0.2)",
        name=f"Радиус {row['Адрес']} ({row['radius_km']} км)"
    ))


fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        center=dict(lat=55.7558, lon=37.6173),
        zoom=10
    ),
    title="Карта с точками и кругами вокруг точек из rent_places_df",
    showlegend=True
)

fig.show()

In [94]:
# Сделаем сравнительный анализ самых популярных напитков (раф, капуччино, американо, эспрессо, латте) среди наших конкурентов
df_surf_menu = lowercase_names(df_surf_menu, "Блюдо")
df_skuratov_beverage = lowercase_names(df_skuratov_beverage, "Блюдо")
df_stars_beverage = lowercase_names(df_stars_beverage, "Блюдо")
df_surf_beverage = df_surf_menu.loc[indexes] # оставили только напитки (вроде)

display(df_surf_beverage)
display(df_skuratov_beverage)
display(df_stars_beverage)

,Блюдо,Цена (руб.)
0,капучино grand,380
1,пончик лимонно-маковый,250
52,лимонад флауэр пауэр grand,430
53,лимонад кейптаун grand,430
54,лимонад дабл элевен grand,430
55,португальский смузи grand,495
56,смузи бали-бум grand,495
57,смузи улувату grand,495
58,айс-латте grand,380
59,айс мэверик бамбл grand,460


,Блюдо,250 мл,350 мл,450 мл,Средняя стоимость
0,латте,NaN,350.0,450.0,400.0
1,флет уайт,350.0,NaN,NaN,350.0
2,матча латте,NaN,410.0,460.0,435.0
3,какао,450.0,500.0,550.0,500.0
4,скуратов эспрессо,300.0,NaN,NaN,300.0
5,сезонный эспрессо,300.0,NaN,NaN,300.0
6,фильтр-кофе,300.0,350.0,400.0,350.0
7,фильтр-кофе с вишней,430.0,480.0,530.0,480.0
8,капучино,350.0,400.0,450.0,400.0
9,ванильный раф,NaN,500.0,550.0,525.0


,Блюдо,Маленький (350 мл),Средний (450 мл),Большой (550 мл),Средняя стоимость
0,капучино,320,345.0,365.0,343.333333
1,латте,365,385.0,415.0,388.333333
2,раф ванилла,370,390.0,420.0,393.333333
3,мокка,420,430.0,455.0,435.000000
4,мокка белый шоколад,395,420.0,445.0,420.000000
5,голден макиато,405,430.0,445.0,426.666667
6,флэт уайт,390,NaN,NaN,390.000000
7,эспрессо 46 мл,230,NaN,NaN,230.000000
8,фильтр-кофе,255,265.0,285.0,268.333333
9,горячий шоколад,370,390.0,415.0,391.666667


In [95]:
keywords = ["раф", "латте", "капучино", "эспрессо", "фильтр", "американо"
]

average_prices_surf = []

for keyword in keywords:
  filtered_df = df_surf_beverage[df_surf_beverage["Блюдо"].str.contains(keyword, na=False)]
  average_price = filtered_df["Цена (руб.)"].mean()
  average_prices_surf.append(average_price)

average_prices_surf[5] = (220+260+300)/3
df_surf_important_position = pd.DataFrame({'Напиток': keywords, 'Средняя стоимость': average_prices_surf})
display(df_surf_important_position)

average_prices_stars = []

for keyword in keywords:
  filtered_df = df_stars_beverage[df_stars_beverage["Блюдо"].str.contains(keyword, na=False)]
  average_price = filtered_df["Средняя стоимость"].mean()
  average_prices_stars.append(average_price)

df_stars_important_position = pd.DataFrame({'Напиток': keywords, 'Средняя стоимость': average_prices_stars})
display(df_stars_important_position)

average_prices_skuratov = []

for keyword in keywords:
  filtered_df = df_skuratov_beverage[df_skuratov_beverage["Блюдо"].str.contains(keyword, na=False)]
  average_price = filtered_df["Средняя стоимость"].mean()
  average_prices_skuratov.append(average_price)

df_skuratov_important_position = pd.DataFrame({'Напиток': keywords, 'Средняя стоимость': average_prices_skuratov})
display(df_skuratov_important_position)

# таблички в порядке
# Surf
# Stars
# Skuratov

,Напиток,Средняя стоимость
0,раф,453.750000
1,латте,405.454545
2,капучино,406.000000
3,эспрессо,380.000000
4,фильтр,293.333333
5,американо,260.000000


,Напиток,Средняя стоимость
0,раф,393.333333
1,латте,418.333333
2,капучино,343.333333
3,эспрессо,317.500000
4,фильтр,268.333333
5,американо,358.333333


,Напиток,Средняя стоимость
0,раф,525.0
1,латте,417.5
2,капучино,400.0
3,эспрессо,300.0
4,фильтр,415.0
5,американо,300.0


In [96]:
import dash
from dash import dcc, html, Input, Output

In [102]:
df_surf_important_position['Сеть'] = 'Surf Coffee'
df_stars_important_position['Сеть'] = 'Stars Coffee'
df_skuratov_important_position['Сеть'] = 'Skuratov'

df_dash = pd.concat([df_surf_important_position, df_stars_important_position, df_skuratov_important_position])

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Средние цены на популярные позиции в кофейнях"),

    dcc.Dropdown(
        id='shop-dropdown',
        options=[
            {'label': 'Surf Coffee', 'value': 'Surf Coffee'},
            {'label': 'Stars Coffee', 'value': 'Stars Coffee'},
            {'label': 'Skuratov', 'value': 'Skuratov'}
        ],
        value='Surf Coffee',
        style={'width': '50%'}
    ),

    dcc.Graph(id='price-graph')
])
@app.callback(
    Output('price-graph', 'figure'),
    [Input('shop-dropdown', 'value')]
)

def update_graph(selected_drink):
    filtered_df = df_combined[df_combined['Напиток'] == selected_drink]
    fig = px.bar(
        filtered_df,
        x='Сеть',
        y='Средняя стоимость',
        title=f'Цены на {selected_drink} в разных сетях',
        labels={'Средняя стоимость': 'Средняя стоимость (руб)', 'Сеть': 'Сеть кофеен'},
        text='Средняя стоимость',
        color='Сеть',  # Разные цвета для каждой сети
        color_discrete_map={
            "Surf Coffee": "#1f77b4",
            "Stars Coffee": "#ff7f0e",
            "Skuratov": "#2ca02c"
        })

    fig.update_traces(
        texttemplate='%{text:.2f} руб',
        textposition='outside',
        width=0.5
    )
    fig.update_layout(
        xaxis_title="Сеть кофеен",
        yaxis_title="Средняя стоимость (руб)",
        showlegend=False,
        bargap=0.2
    )
    return fig


if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [103]:
df_surf_important_position["Сеть"] = "Surf Coffee"
df_stars_important_position["Сеть"] = "Stars Coffee"
df_skuratov_important_position["Сеть"] = "Skuratov"

df_combined = pd.concat([df_surf_important_position, df_stars_important_position, df_skuratov_important_position])

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Сравнение цен на напитки в сетях кофеен"),

    dcc.Dropdown(
        id='drink-dropdown',
        options=[{'label': drink, 'value': drink} for drink in df_combined['Напиток'].unique()],
        value='капучино',
        clearable=False
    ),

    dcc.Graph(id='price-graph')
])

@app.callback(
    Output('price-graph', 'figure'),
    [Input('drink-dropdown', 'value')]
)
def update_graph(selected_drink):

    filtered_df = df_combined[df_combined['Напиток'] == selected_drink]

    fig = px.bar(
        filtered_df,
        x='Сеть',
        y='Средняя стоимость',
        title=f'Цены на {selected_drink} в разных сетях',
        labels={'Средняя стоимость': 'Средняя стоимость (руб)', 'Сеть': 'Сеть кофеен'},
        text='Средняя стоимость'
    )

    fig.update_traces(texttemplate='%{text:.2f} руб', textposition='outside')

    return fig

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [105]:
# гпт помогает мне решить проблему 403
!pip install dash
!pip install dash-bootstrap-components
!pip install pyngrok

from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from pyngrok import ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 5.8 MB/s eta 0:00:00


ERROR:pyngrok.process.ngrok:t=2025-03-22T13:50:48+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-22T13:50:48+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-22T13:50:48+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [111]:

NGROK_AUTH_TOKEN = "2uftey0oBf5DwsuIRXzV5KoDcAn_6kH4WzXBpDnYMEHeLCe79"  # Только токен
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

df_combined = pd.concat([df_surf_important_position, df_stars_important_position, df_skuratov_important_position])

#app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Средние цены на популярные позиции в кофейнях"),

    dcc.Dropdown(
        id='shop-dropdown',
        options=[
            {'label': 'Surf Coffee', 'value': 'Surf Coffee'},
            {'label': 'Stars Coffee', 'value': 'Stars Coffee'},
            {'label': 'Skuratov', 'value': 'Skuratov'}
        ],
        value='Surf Coffee',
        style={'width': '50%'}
    ),

    dcc.Graph(id='price-graph')
])
@app.callback(
    Output('price-graph', 'figure'),
    [Input('shop-dropdown', 'value')]
)

def update_graph(selected_drink):
    filtered_df = df_combined[df_combined['Напиток'] == selected_drink]
    fig = px.bar(
        filtered_df,
        x='Сеть',
        y='Средняя стоимость',
        title=f'Цены на {selected_drink} в разных сетях',
        labels={'Средняя стоимость': 'Средняя стоимость (руб)', 'Сеть': 'Сеть кофеен'},
        text='Средняя стоимость',
        color='Сеть',  # Разные цвета для каждой сети
        color_discrete_map={
            "Surf Coffee": "#1f77b4",
            "Stars Coffee": "#ff7f0e",
            "Skuratov": "#2ca02c"
        })

    fig.update_traces(
        texttemplate='%{text:.2f} руб',
        textposition='outside',
        width=0.5
    )
    fig.update_layout(
        xaxis_title="Сеть кофеен",
        yaxis_title="Средняя стоимость (руб)",
        showlegend=False,
        bargap=0.2
    )
    return fig



if __name__ == '__main__':
    app.run(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.